In [1]:
import plotly.graph_objects as go
from pandas import read_csv
import csv
import math
import numpy as np
import random
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from numpy.linalg import eig
from pyquaternion import Quaternion
from random import randrange
import sympy as sym
import script as rt

# Вчитување на податоци

In [2]:
a = 6378137
e = 0.00699437999014
with open('Senzori_new.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    x_s = []
    y_s = []
    z_s = []
    time_senzori = []
    i = 0
    for row in readCSV:
        if i > 0:
            latitude = float(row[2])
            longitude = float(row[3])
            altitude = float(row[20])
            N = (a / (math.sqrt(1 - e * (math.sin(latitude)) ** 2)))
            x = ((N + altitude) * math.cos(latitude) * math.cos(longitude))/10
            y = ((N + altitude) * math.cos(latitude) * math.sin(longitude))/10
            z = ((N * (1 - e) + altitude) * math.sin(latitude))/10
            t = int(row[0])
            time_senzori.append(t)
            x_s.append(x)
            y_s.append(y)
            z_s.append(z)
        i = i + 1

In [3]:
with open('SLAM_new.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    x_s1 = []
    y_s1 = []
    z_s1 = []
    xf, yf, zf = [], [], []
    time_SLAM = []
    i = 0
    fps = 30
    for row in readCSV:
        if i >= 1:
            x1 = float(row[6])
            y1 = float(row[7])
            z1 = float(row[8])
            xx1 = float(row[18])
            yy1 = float(row[19])
            zz1 = float(row[20])
            frame = int(row[1])
            m = frame/fps
            t = "{0:0.1f}".format(m)
            time_SLAM.append(t)
            xf.append(xx1)
            yf.append(yy1)
            zf.append(zz1)
            x_s1.append(x1)
            y_s1.append(y1)
            z_s1.append(z1)
        i = i + 1   

# Издвојување на податоци(земање само оние кои се во ист временски момент)

In [4]:
def times(time): # funkcija sto go doveduva vremeto da pocne od nula
    ktime = []
    init_time = time[0]
    for i in range(len(time)):
        t = (time[i] - init_time)/1000
        ktime.append(t)
    return ktime

time_sensors = times(time_senzori)

In [5]:
def podatoci(time_sensor, time_slam, x_s, x_s1, y_s, y_s1, z_s, z_s1):
    xp = []
    xp1 = []
    yp = []
    yp1 = []
    zp = []
    zp1 = []
    sumx1 = 0
    sumy1 = 0
    sumz1 = 0
    sumx = 0
    sumy = 0
    sumz = 0
    for i in range(len(time_slam)):  #podatoci od slam
        ti1 = float(time_slam[i])
        x1 = x_s1[i]
        y1 = y_s1[i]
        z1 = z_s1[i]
        sumx1 = sumx1 + x1
        sumy1 = sumy1 + y1
        sumz1 = sumz1 + z1
        xp1.append(x1)
        yp1.append(y1)
        zp1.append(z1)
        for i in range(len(time_sensor)):  # podatoci od SLAM
            ti = float(time_sensor[i])
            k = ti1 - ti
            if k == 0:
                x = x_s[i]
                y = y_s[i]
                z = z_s[i]
                sumx = sumx + x
                sumy = sumy + y
                sumz = sumz + z
                xp.append(x)
                yp.append(y)
                zp.append(z)
    sredx1 = sumx1/len(xp1)
    sredy1 = sumy1/len(yp1)
    sredz1 = sumz1/len(zp1)
    sredx = sumx/len(xp)
    sredy = sumy/len(yp)
    sredz = sumz/len(zp)

    return xp, xp1, yp, yp1, zp, zp1, sredx, sredy, sredz, sredx1, sredy1, sredz1      

xs, xs1, ys, ys1, zs, zs1, sredxP, sredyP, sredzP, sredxS, sredyS, sredzS = podatoci(time_sensors, time_SLAM, x_s, x_s1, y_s, y_s1, z_s, z_s1)

# Транслирање и скалирање

За да бидат во ист координатен систем со ист размер

In [6]:
def translation(x, y, z, xsred, ysred, zsred):
    i = 0
    xk = []
    yk = []
    zk = []
    xsum = 0 
    ysum = 0
    zsum = 0
    for i in range(len(x)):
        A = np.array([[x[i]],
                [y[i]],
                [z[i]], 
                [1]])
        B = np.array([[1, 0, 0, -xsred],
                [0, 1, 0, -ysred],
                [0, 0, 1, -zsred], 
                [0, 0, 0, 1]])
        C = B @ A
        x1 = C[0][0]
        y1 = C[1][0]
        z1 = C[2][0]
        xsum = xsum + x1
        ysum = ysum + y1
        zsum = zsum + z1
        xk.append(x1)
        yk.append(y1)
        zk.append(z1)
        i = i + 1
    tsredx = xsum/len(x)
    tsredy = ysum/len(y)
    tsredz = zsum/len(z)
    return xk, yk, zk, tsredx, tsredy, tsredz

xP, yP, zP, tsredx, tsredy, tsredz = translation(xs, ys, zs, sredxP, sredyP, sredzP) # translacija na tockite od senzori

xS, yS, zS, tsredxS, tsredyS, tsredzS = translation(xs1, ys1, zs1, sredxS, sredyS, sredzS)  # translacija na tockite od SLAM

In [7]:
def suma_rastojanija(x,y,z,xs,ys,zs):
    D = 0
    for i in range(len(x)):
        D = D + math.sqrt((x[i] - xs)**2 + (y[i] - ys)**2 + (z[i] - zs)**2)
    return D
d = suma_rastojanija(xP, yP, zP, tsredx, tsredy, tsredz)
dS = suma_rastojanija(xS, yS, zS, tsredxS, tsredyS, tsredzS)
scal = d /dS

In [8]:
def scale(x, y, z, sx, sy, sz):
    xsk, ysk, zsk = [], [], []
    i = 0
    for i in range(len(x)):
        A = np.array([[x[i]],
                [y[i]],
                [z[i]], 
                [1]])

        B = np.array([[sx, 0, 0, 0],
                    [0, sy, 0, 0],
                    [0, 0, sz, 0],
                    [0, 0, 0, 1]])
        C = B@A
        xs = C[0][0]
        ys = C[1][0]
        zs = C[2][0]
        xsk.append(xs)
        ysk.append(ys)
        zsk.append(zs)
        i = i + 1
    
    return xsk, ysk, zsk

xsk, ysk, zsk = scale(xS, yS, zS, scal, scal, scal)

In [9]:
w = np.ones(len(xP))
sensor_data = rt.hpoint3(xP, yP, zP, w)
sensor_data = np.array(sensor_data, dtype=np.float)

slam_scale = rt.hpoint3(xsk, ysk, zsk, w)
slam_scale = np.array(slam_scale, dtype=np.float)


# Ротациона матрица со Gradient descent

In [10]:
fig = go.FigureWidget()
#fig.update_layout(xaxis={'range': [-7, 6]}, yaxis={'range': [1, 13]}),
fig.update_layout(
    scene = dict(
        xaxis = dict(),
                     yaxis = dict(),
                     zaxis = dict(),),)
fig.add_trace(go.Scatter3d(x=sensor_data[0, :], y=sensor_data[1, :], z=sensor_data[2, :], mode='markers', name='Сензори',
                         marker={'size': 5, 'symbol': 'x', 'color': np.arange(sensor_data.shape[1])}))
fig.add_trace(go.Scatter3d(x=slam_scale[0, :], y=slam_scale[1, :], z=slam_scale[2, :], mode='markers', name='SLAM', 
                          marker={'size': 5, 'color': np.arange(slam_scale.shape[1])}))

fig

FigureWidget({
    'data': [{'marker': {'color': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13…

In [11]:
def step_gradient(sensor_points, slam_points, transform, learning_rate):
    """
    Чекор со gradient descent.
    
    sensor_points, slam_points: се податоците кои ќе ни послужат за пресметка на грешката.
    transform: е матрицата на трансформација чии членови ги бараме во оваа функција
    learning_rate: стапката на учење
    """
    x_gradient, y_gradient, z_gradient, rx_g, ry_g, rz_g = 0, 0, 0, 0, 0, 0
    new_sensor_points = (transform @ slam_points).T
    r_vector = sensor_points.T - new_sensor_points 
    r_x_vector, r_y_vector, r_z_vector = r_vector[:, 0], r_vector[:, 1], r_vector[:, 2] 
    m = len(r_vector)
    for r_x, r_y, r_z, sensor_point in zip(r_x_vector, r_y_vector, r_z_vector, sensor_points):
        x_gradient += r_x 
        y_gradient += r_y  
        z_gradient += r_z
        rx_g += -sensor_point[2] * r_y + sensor_point[1] * r_z    # го пресметуваме градиентот за аголот
        ry_g +=  sensor_point[2] * r_x - sensor_point[0] * r_z    
        rz_g += -sensor_point[1] * r_x + sensor_point[0] * r_y    
    dx = -learning_rate * x_gradient / m  
    dy = -learning_rate * y_gradient / m  
    dz = -learning_rate * z_gradient / m
    dalpha = -learning_rate/10 * rx_g / m   
    dbeta = -learning_rate/10 * ry_g / m
    dgama = -learning_rate/10 * rz_g / m
    if dalpha>=0.000001 or dbeta>=0.000001 or dgama>=0.000001:
        delta_transform = np.array(rt.hrotation3('x', dalpha) * rt.hrotation3('y', dbeta) * rt.hrotation3('z', dgama), dtype=np.float)
        transform = delta_transform @ transform
    else:
        return transform, new_sensor_points

    # претходните 3 чекори по секој степен на слобода ги соединуваме во една трансформациона матрица
    #delta_transform = np.array(rt.hrotation3('x', dalpha) * rt.hrotation3('y', dbeta) * rt.hrotation3('z', dgama), dtype=np.float)
    # и конечно го правиме чекорот
    #transform = delta_transform @ transform
    # на крајот ја враќаме обновената вредност на transform и предвидената вредност за изместените податоци за да ги исцртаме
    return transform, new_sensor_points

def run(fig, sensor_points, slam_points):
    learning_rate = -0.01
    num_iterations = 10000
    # почнуваме со случајно избрани вредности за alpha, beta, gama
    alpha, beta, gama = 0, 0, 0
    transform = np.array(rt.hrotation3('x', alpha) * rt.hrotation3('y', beta) * rt.hrotation3('z', gama), dtype=np.float)
    for i in range(num_iterations):
        transform, new_sensor_points = step_gradient(sensor_points, slam_points, transform, learning_rate)
        # цртаме на секоја стота итерација
        if i % 100 == 0:
            with fig.batch_update():
                fig.data[1].x, fig.data[1].y, fig.data[1].z = new_sensor_points[:, 0], new_sensor_points[:, 1], new_sensor_points[:, 2]
                fig.data[1].marker.color = np.arange(len(new_sensor_points))
    
    return transform, new_sensor_points 

#final_transform, sensor_d = run(fig, sensor_data, slam_scale)
#final_transform

array([[ 0.4460304 ,  0.89297936, -0.06037173,  0.        ],
       [-0.6138504 ,  0.25612608, -0.74671756,  0.        ],
       [-0.65134059,  0.37011794,  0.66239576,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

#                                    Симулација за во реално време

In [12]:
def runn(fig, sensor_points, slam_points, transform):
    learning_rate = -0.01
    num_interations = 1000
    for i in range(num_interations):
        transform, new_sensor_points = step_gradient(sensor_points, slam_points, transform, learning_rate)
        if i % 100 == 0:
            with fig.batch_update():
                #fig.data[0].x, fig.data[0].y, fig.data[0].z = slam_points[0, :], slam_points[1, :], slam_points[2, :]
                #fig.data[0].marker.color = np.arange(len(slam_points[0, :]))
                fig.data[1].x, fig.data[1].y, fig.data[1].z = new_sensor_points[:, 0], new_sensor_points[:, 1], new_sensor_points[:, 2]
                fig.data[1].marker.color = np.arange(len(new_sensor_points))
    return transform, new_sensor_points

def real_time(fig, sensor_data, slam_scale):
    alpha, beta, gama = 0, 0, 0
    #почетна трансформациона матрица
    transform = np.array(rt.hrotation3('x', alpha) * rt.hrotation3('y', beta) * rt.hrotation3('z', gama), dtype=np.float)
    for i in range(len(sensor_data[0, :])):
        # земи ги првите 5 точки и во секој нареден чекор земај ги наредните 3
        i = 5 + i*3
        sensor_dataa = sensor_data[:4, :i] 
        slam_scalee = slam_scale[:4, :i]
        #print(transform)
        # повикај  ГД за секоја интерација
        transform, sensor_d = runn(fig, sensor_dataa, slam_scalee, transform)
        #print(transform)
    return transform

In [13]:
fig = go.FigureWidget()
fig.update_layout(
    scene = dict(
        xaxis = dict(),
                     yaxis = dict(),
                     zaxis = dict(),),)
fig.add_trace(go.Scatter3d(x=sensor_data[0, :], y=sensor_data[1, :], z=sensor_data[2, :], mode='markers', name='Сензори',
                         marker={'size': 5, 'symbol': 'x', 'color': np.arange(sensor_data.shape[1])}))
fig.add_trace(go.Scatter3d(x=slam_scale[0, :], y=slam_scale[1, :], z=slam_scale[2, :], mode='markers', name='SLAM', 
                          marker={'size': 5, 'color': np.arange(slam_scale.shape[1])}))

fig

FigureWidget({
    'data': [{'marker': {'color': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13…

In [14]:
real_time(fig, sensor_data, slam_scale)

array([[ 0.32552009,  0.89011443,  0.31895606,  0.        ],
       [ 0.36324929,  0.19371619, -0.91133089,  0.        ],
       [-0.87297573,  0.41251707, -0.26027492,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])